In [104]:
import pandas as pd

df = pd.read_csv("../data/train.csv")
print(len(df))

2800


### 한자 탐색

In [105]:
import pandas as pd
import re
from collections import Counter

# 한자 추출 함수 정의
def extract_hanza(text):
    # CJK 통합 한자 범위(U+4E00 ~ U+9FFF)에 해당하는 문자 추출
    return "".join(re.findall("[\u4e00-\u9fff]", text))

# 문자 빈도 축정
def get_char_frequency(text_list):
    # 모든 문자를 하나의 리스트로 합치기
    chars = [char for text in text_list for char in text]
    # Counter를 사용하여 빈도 계산
    char_counts = Counter(chars)
    # 빈도순으로 정렬
    return pd.Series(char_counts).sort_values(ascending=False)

In [106]:
# 한자 빈도수 측정
df["hanza"] = df["text"].apply(extract_hanza)

char_frequency = get_char_frequency(df["hanza"])
print(char_frequency)
hanza = list(char_frequency.keys())
print(len(hanza), hanza)

美    79
北    55
中    43
朴    36
靑    24
日    21
與    20
文     9
英     7
野     6
佛     5
伊     5
獨     4
反     4
前     3
軍     3
硏     3
對     2
外     2
社     2
黃     2
亞     2
韓     2
株     1
車     1
崔     1
院     1
金     1
丁     1
小     1
和     1
企     1
安     1
展     1
檢     1
親     1
銀     1
證     1
先     1
父     1
南     1
詩     1
家     1
大     1
印     1
阿     1
故     1
州     1
重     1
dtype: int64
49 ['美', '北', '中', '朴', '靑', '日', '與', '文', '英', '野', '佛', '伊', '獨', '反', '前', '軍', '硏', '對', '外', '社', '黃', '亞', '韓', '株', '車', '崔', '院', '金', '丁', '小', '和', '企', '安', '展', '檢', '親', '銀', '證', '先', '父', '南', '詩', '家', '大', '印', '阿', '故', '州', '重']


In [107]:
df_hanza = df[df["hanza"].str.len()!=0]
print(len(df_hanza))
df_hanza = df_hanza[["ID", "text", "target"]]
#df_hanza.to_csv("hanza.csv",index=False)
df_hanza.head()

313


,ID,text,target
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
50,ynat-v1_train_00050,"美MBA[여성 비율x계속 x가4주$E19a대 U입생 중Ym,%",6
58,ynat-v1_train_00058,한#M2 !는 유`8 치료제 오B솔 美 임{ 3a 본격화,6


### 한자 전부 제거

In [108]:
def remove_chars_from_text(df, char_list):
    pattern = "[" + re.escape("".join(char_list)) + "]"
    df["filtered_text"] = df["filtered_text"].str.replace(pattern, "", regex=True)
    return df

In [109]:
df["filtered_text"] = df["text"].copy()
df = remove_chars_from_text(df, hanza)
df = df.drop(columns=["hanza"])
df.head()

,ID,text,target,filtered_text
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰.국DLwo 로L3한N% 회장 2 T0&}송=
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,"m 김정) 자주통일 새,?r열1나가야1보"
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,pI대선I앞두고 R2fr단 발] $비해 감시 강화


### 자주 쓰이는 특수문자 탐색

In [110]:
#영어, 숫자, 한글 제외
def extract_special(text):
    return "".join(re.findall(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣]", text))

In [111]:
# 특수문자 빈도수 측정
special = df["filtered_text"].apply(extract_special)
char_frequency = get_char_frequency(special)
filtered_chars = char_frequency[char_frequency >= 100]
zazu_special = list(filtered_chars.keys())
print(len(zazu_special),zazu_special)

34 ['…', '.', '·', '%', '"', '-', '(', '|', '?', ',', '}', ':', '&', '_', '{', '~', '#', '\\', '*', ')', '$', '=', '+', '`', ';', "'", '!', '@', '<', '/', '>', '[', ']', '^']


### Case1. 말줄임표는 띄어쓰기로 치환

In [112]:
# df["filtered_text"] = df["filtered_text"].str.replace("…", " ")
# # df["filtered_text"] = df["filtered_text"].str.replace(
# #     "|".join(["…", "·", "/"]), " ", regex=True
# # )

### Case2. 100번 이상 사용된 특수문자 제거

In [113]:
df = remove_chars_from_text(df, zazu_special)
df["filtered_list"] = df["filtered_text"].apply(str.split)
df.head()

,ID,text,target,filtered_text,filtered_list
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정i 파1 미사z KT 이용기간 2e 단 Q분종U2보,"[정i, 파1, 미사z, KT, 이용기간, 2e, 단, Q분종U2보]"
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰국DLwo 로L3한N 회장 2 T0송,"[K찰국DLwo, 로L3한N, 회장, 2, T0송]"
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,m 김정 자주통일 새r열1나가야1보,"[m, 김정, 자주통일, 새r열1나가야1보]"
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,"[갤노트8, 주말, 27만대, 개통시장은, 불법, 보조금, 얼룩]"
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,pI대선I앞두고 R2fr단 발 비해 감시 강화,"[pI대선I앞두고, R2fr단, 발, 비해, 감시, 강화]"


### 형태소 분석기를 통한 한글+영문+숫자+특문 혼합 단어 탐지

In [114]:
# !pip install KoNLPy

In [115]:
# 분석기 임포트
from konlpy.tag import *

# 각 분석기 객체 생성
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [116]:
df["hannanum"] = df["filtered_list"].apply(lambda li: [hannanum.pos(e) for e in li])
df["kkma"] = df["filtered_list"].apply(lambda li: [kkma.pos(e) for e in li])
df["komoran"] = df["filtered_list"].apply(lambda li: [komoran.pos(e) for e in li])
df["okt"] = df["filtered_list"].apply(lambda li: [okt.pos(e) for e in li])

In [117]:
from collections import Counter

# 문장에서 조건 탐지
def check_tags_count_V1(morpheme_lists):
    # 단어에서 조건 탐지
    def check_tags_count_word(sublist):
        tag_counts = Counter(tag for _, tag in sublist)

        alpha = tag_counts["Alpha"]
        punctuation = tag_counts["Punctuation"]
        number = tag_counts["Number"]
        rest = sum(tag_counts.values()) - (alpha + punctuation + number)

        # 조건 리스트
        conditions = [
            alpha >= 1 and punctuation >= 1 and number >= 1,
            alpha >= 2,
            punctuation >= 2,
            # number >= 2,
        ]

        return any(conditions)

    # 하나의 단어라도 조건을 만족하면 True
    return any(check_tags_count_word(sublist) for sublist in morpheme_lists)


# 데이터프레임에 적용
df["check"] = df["okt"].apply(check_tags_count_V1)

In [118]:
df_morph_condition_V1 = df[df["check"]==True]
df_morph_condition_V1 = df_morph_condition_V1[["ID", "target", "text"]]
df_morph_condition_V1.to_csv("df_morph_condition_V1.csv",index=False)
print(len(df_morph_condition_V1))
df_morph_condition_V1.head()

1120


,ID,target,text
0,ynat-v1_train_00000,4,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1,ynat-v1_train_00001,3,K찰.국DLwo 로L3한N% 회장 2 T0&}송=
4,ynat-v1_train_00004,6,pI美대선I앞두고 R2fr단 발] $비해 감시 강화
10,ynat-v1_train_00010,5,oi 매력 R모h츠a열#w3약 >l·주가 고Q/진
18,ynat-v1_train_00018,0,개R전 연w정연H 작가


### morph 패턴 V1 기반 1118개 노이즈 탐지, 2개 오탐지. 남은 데이터: 1680개. 남은 노이즈 : 482개

In [120]:
df = df[df["check"] == False]
len(df)

1680

In [121]:
# 문장 통째로 탐색
def check_tags_count_V2(morpheme_lists):
    # 태그 카운트
    all_tags = [tag for sublist in morpheme_lists for _, tag in sublist]
    tag_counts = Counter(all_tags)

    alpha = tag_counts["Alpha"]
    punctuation = tag_counts["Punctuation"]
    number = tag_counts["Number"]
    rest = sum(tag_counts.values()) - (alpha + punctuation + number)

    # 조건 리스트
    conditions = []
    # conditions.append(punctuation >= 1)
    conditions.append(alpha + punctuation + number >= 6)
    return all(conditions)

# 데이터프레임에 적용
df["check"] = df["okt"].apply(check_tags_count_V2)

In [122]:
df_morph_condition_V2 = df[df["check"] == True]
df_morph_condition_V2 = df_morph_condition_V2[["ID", "target", "text"]]
df_morph_condition_V2.to_csv("df_morph_condition_V2.csv", index=False)
print(len(df_morph_condition_V2))
df_morph_condition_V2.head()

101


,ID,target,text
13,ynat-v1_train_00013,4,아이`XSI수리0* b대`…맥3 디dF레< 41/'
14,ynat-v1_train_00014,2,"문/인 당2 4nS 민관2동7사위 /""X보 철거tt"
54,ynat-v1_train_00054,4,"찍W ,fK는 즐거m T졌다…5개 카메: LG V40 z큐"
58,ynat-v1_train_00058,6,한#M2 !는 유`8 치료제 오B솔 美 임{ 3a 본격화
94,ynat-v1_train_00094,1,"멀티골 j순형 u로축구 pB그1 1""\운드FXVP"


### morph 패턴 V2 기반 96개 노이즈 탐지, 5개 오탐지. 남은 데이터: 1579개. 남은 노이즈 : 386개

In [123]:
df = df[df["check"] == False]
len(df)

1579

### 한글 샌드위치 패턴
한글 사이에 낀 문자 중 (한글, 영어 대문자, 숫자, 관계 표현 특수문자)를 제외한 문자들 탐지

In [128]:
def detect_sandwich_pattern(text_list):
    return any(
        bool(re.search(r"[가-힣]+[^A-Z가-힣0-9ㆍ><∼→←↑↓↔]+[가-힣]+", str(item)))
        for item in text_list
    )

df["sandwich"] = df["filtered_list"].apply(detect_sandwich_pattern)
sandwich_df = df[df["sandwich"]==True]
not_sandwich_df = df[df["sandwich"] == False]

In [131]:
print(len(sandwich_df),len(not_sandwich_df))

121 1458


In [132]:
sandwich_df = sandwich_df[["ID", "target", "text"]]
sandwich_df.to_csv("sandwich_df.csv", index=False)

### 샌드위치 패턴 기반 121개 노이즈 탐지. 0개 오탐지. 남은 데이터: 1458개. 남은 노이즈: 265개

In [133]:
df = not_sandwich_df
len(df)

1458

### 문자 비율 기반 노이즈 탐색

In [144]:
import re

# 한글.숫자,영어 대문자 기반의 노이즈 탐색
def calculate_noise_ratio(text):
    if len(text)==0:
        return 0
    return round((len(re.findall(r"[^A-Z0-9가-힣\s]", text)) / len(text)), 4)

In [145]:
df.loc[:, "noise_ratio"] = df["filtered_text"].apply(calculate_noise_ratio)

In [147]:
high_special_character = df[df["noise_ratio"] >= 0.09]
print(len(high_special_character))
high_special_character = high_special_character.sort_values(
    by=["noise_ratio"], axis=0, ascending=True
)

high_special_character = high_special_character[["ID", "target", "text"]]
high_special_character = high_special_character.sort_values(by="ID", ascending=True)
high_special_character.to_csv("high_special_character_0.09.csv", index=False)
high_special_character.head()

114


,ID,target,text
6,ynat-v1_train_00006,1,프로야구~롯TKIAs광주 경기 y천취소
30,ynat-v1_train_00030,4,해외로밍 m금폭탄 n동차단 더 빨$진다
53,ynat-v1_train_00053,3,코로나 r대^등교)모습
101,ynat-v1_train_00101,1,~학농구리- 8일 고려대중앙대 경pt 개막
136,ynat-v1_train_00136,2,xW리 a)엔 예비후0V사전여론조사 결과 유출 논c


In [148]:
less_special_character = df[df["noise_ratio"]<0.09]
print(len(less_special_character))
less_special_character = less_special_character.sort_values(
    by=["noise_ratio"], axis=0, ascending=False
)
less_special_character = less_special_character[["ID", "target", "text"]]
less_special_character.to_csv("rest.csv", index=False)
less_special_character.head()

1365


,ID,target,text
1201,ynat-v1_train_01201,2,여야 ~종인 개헌특l 제안에 ~갈v -응&험로 예'
615,ynat-v1_train_00615,4,"티맥, 인d·AI a업지능으로 즈니(·생활 혁신"
2604,ynat-v1_train_02604,6,브_질A언론 {세프 V통. 탄핵rM능성 더 커j
2592,ynat-v1_train_02592,2,[ '대통령재벌 총수 u공& 면담 경위 수사종b
2473,ynat-v1_train_02473,1,v준우 \인 성적 필요 *어…5위 F[만 g각한다


### 문자 비율 기반 110개 노이즈 탐지. 4개 오탐지. 남은 데이터: 1365개. 남은 노이즈: 155개

In [149]:
df = df[df["noise_ratio"] < 0.09]
len(df)

1365

In [150]:
df.head()

,ID,text,target,filtered_text,filtered_list,hannanum,kkma,komoran,okt,check,sandwich,noise_ratio
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,"[갤노트8, 주말, 27만대, 개통시장은, 불법, 보조금, 얼룩]","[[(갤노트8, N)], [(주말, N)], [(27만대, N)], [(개통시장, ...","[[(개, VV), (ㄹ, ETD), (노트, NNG), (8, NR)], [(주말...","[[(개, VV), (ㄹ, ETM), (노트, NNP), (8, SN)], [(주말...","[[(갤, Verb), (노트, Noun), (8, Number)], [(주말, N...",False,False,0.0
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,"[성인, 6명, 중, 1명꼴, 배우자연인, 빚, 떠안은, 적, 있다]","[[(성인, N)], [(6명, N)], [(중, N)], [(1명, N), (꼴,...","[[(성인, NNG)], [(6, NR), (명, NNM)], [(중, NNG)],...","[[(성인, NNG)], [(6, SN), (명, NNB)], [(중, NNB)],...","[[(성인, Noun)], [(6, Number), (명, Noun)], [(중, ...",False,False,0.0
7,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,4,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,"[아가메즈, 33득점, 우리카드, KB손해보험, 완파3위, 굳]","[[(아가메즈, N)], [(33득점, N)], [(우리카드, N)], [(KB, ...","[[(아가, NNG), (메, NNG), (즈, UN)], [(33, NR), (득...","[[(아가메즈, NA)], [(33, SN), (득점, NNP)], [(우리카드, ...","[[(아가, Noun), (메, Noun), (즈, Modifier)], [(33,...",False,False,0.0
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6,대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,"[대통령, 얼마나, 많이, 놀라셨어요경주, 지진현장, 방문종합]","[[(대통령, N)], [(얼마나, M)], [(많, P), (이, X)], [(놀...","[[(대통령, NNG)], [(얼마나, MAG)], [(많이, MAG)], [(놀라...","[[(대통령, NNG)], [(얼마나, MAG)], [(많이, MAG)], [(놀라...","[[(대통령, Noun)], [(얼마나, Noun)], [(많이, Adverb)],...",False,False,0.0
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,"[듀얼심, 아이폰, 하반기, 출시설, 솔솔알뜰폰, 기대감]","[[(듀얼심, N)], [(아이폰, N)], [(하반기, N)], [(출시설, N)...","[[(듀얼, NNG), (심, NNG)], [(아이, NNG), (폰, NNG)],...","[[(듀얼심, NA)], [(아이폰, NNP)], [(하반기, NNG)], [(출시...","[[(듀얼, Noun), (심, Noun)], [(아이폰, Noun)], [(하반기...",False,False,0.0


# 오탐지 데이터
### morph_condition V1
```
ynat-v1_train_00858,5,한국MS 윈도10 IoT 에디션 출시…B2B 공략
ynat-v1_train_01717,6,中 베이징서 H7N9 조류인플루엔자 환자 또 발생
```

### morph_condition V2
```
ynat-v1_train_00982,4,회전 카메라 탑재한 갤럭시A80 SKT 단독출시…59만9천500원
ynat-v1_train_01178,1,통신3사 5G 가입자 경쟁 불붙었다…LG V50 지원금 최대 77만원
ynat-v1_train_01225,5,세계 첫 5G폰 갤럭시S10 5G 출고가 139만7천원 확정종합
ynat-v1_train_01776,5,LGU 5G SA 상용화 준비…SA 기술 NSA 코어 장비에 연동 검증
ynat-v1_train_01873,3,LG전자 스마트폰사업 10분기 적자…3Q 영업손실 3천753억종합
```

### sandwich
```
```

### 문자 비율 기반 정규표현식
```
ynat-v1_train_00191,4,MLB.com 다저스 3년 연속 WS 유력하지만 우승은 글쎄…
ynat-v1_train_00741,6,게시판 SKT AI 콘퍼런스 ai.x 2019 개최
ynat-v1_train_02249,0,클린턴재단 후폭풍인가…힐러리 vs 트럼프 지지율 3%p로 좁혀져
ynat-v1_train_02399,2,재단법인화 tbs 초대 대표에 이강택 현 교통방송 대표
```